In [1]:
import numpy as np
import george
from george import kernels
from scipy.optimize import minimize
from time import time
import argparse

In [2]:
rname="first"
wname="first"
solver="COBYLA"
verbose=True

In [3]:
coords=np.load("./etgrid/"+rname+"_coords.npy")
eigenseds=np.load("./etgrid/"+rname+"_eigenseds.npy")
weights=np.load("./etgrid/"+rname+"_weights.npy")
pcamean=np.load("./etgrid/"+rname+"_mean.npy")

sedsflat=np.ndarray.flatten(np.load('./etgrid/seds.npy')[:,100:500])

In [4]:
yerrs=[]
for i in range(16):
    yerrs.append([x*0.01 for x in weights[i]])
    
initvecs=[]
for i in range(16):
    initvecs.append([ 6.33043185, 18.42068074,  0.,  0., -0.4462871 , -5.05145729, -1.38629436, 
                     -2.4079456086518722, -2.4079456086518722,  -3.2188758248682006, -3.21887582, 
                     -2.77258872, -2.77258872,  1.38629436,  2.19722458, 0.8109302162163288])


In [5]:
kernel = 16*kernels.ExpSquaredKernel(15**2,ndim=15,axes=0)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=1)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=2)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=3)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=4)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=5)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=6)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=7)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=8)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=9)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=10)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=11)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=12)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=13)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=14) 
blankhodlr=george.GP(kernel,solver=george.HODLRSolver)


In [6]:
def F_chisq_quiet(hp,gp):
    hyperparams=np.array(hp).reshape(16,16)
    preds=[]
    for i in range(len(weights)):  # same covfunc for each weight and the sample mean
        gp.set_parameter_vector(hyperparams[i])
        if verbose==True:
            print("weight #"+str(i))
            print(gp.get_parameter_vector())
        gp.compute(coords,yerrs[i])
        pred, pred_var = gp.predict(weights[i], coords, return_var=True)
        preds.append(pred)
    reconst_SEDs=[]
    for i in range(3825):
        reconst=np.dot(np.array(preds)[:,i][0:15],eigenseds[0:15]) + pcamean + np.array(preds)[:,i][15]
        reconst_SEDs.append(reconst)
    allsedsflat=np.ndarray.flatten(np.array(reconst_SEDs))
    chisq=np.sum((sedsflat-allsedsflat)**2/0.1)
    if verbose==True:
        print(chisq)
    return chisq

def chisq(p):
    return F_chisq_quiet(p,blankhodlr)


In [ ]:
minimize(chisq,initvecs,method=solver)

In [7]:
F_chisq_quiet(initvecs,blankhodlr)

weight #0
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
weight #1
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
weight #2
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
weight #3
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
weight #4
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.3862

inf